In [7]:
import gensim
from gensim import matutils, corpora
from gensim.models.ldamodel import LdaModel
import pandas as pd
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis
import pyLDAvis.gensim

import snowball

In [8]:
tweet_data = pd.read_csv("CancerReport-clean-all-data-en.txt", delimiter="\t")

/anaconda/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
tweet_data.shape

(50410, 40)

In [10]:
tweet_data.head()

,tweet_id,tweet_text,tweet_created_at,in_reply_to_status_id_str,in_reply_to_screen_name,retweet_count,favorite_count,machine_translated_language,geo_lat,geo_long,...,retweet_created_at,retweet_screen_name,retweet_user_created_at,retweet_person_name,retweet_statuses_count,retweet_friends_count,retweet_followers_count,retweet_urls,retweet_hashtag_text,retweet_usermentions_screen_name
0,185077342425714688,RT @jockosports: 2 more days until my #colonos...,2012-03-28 14:54:11,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185081180222263297,My arm is so sore :( had a HPV injection at sc...,2012-03-28 15:09:26,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,185096288109461507,MsRhea Mike HEALTHY Wednesday Awareness mak...,2012-03-28 16:09:28,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185108334121193472,@xomorgyyxo I asked him three times if it was ...,2012-03-28 16:57:20,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185117800669523968,I have been poked and prodded today.and my boo...,2012-03-28 17:34:57,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
tweets = tweet_data["tweet_text"].tolist()

In [12]:
tweets[0]

'RT @jockosports: 2 more days until my #colonoscopy. I hope to raise awareness of colon cancer and prevention.  Follow my journey on #402 ...'

In [13]:
# this will be *all* tweets; independent of the hashtag
toked_tweets = snowball.build_gensim_corpus(tweets)

In [14]:
print(list(toked_tweets[0]))

['rt', 'days', 'colonoscopy', 'hope', 'raise', 'awareness', 'colon', 'cancer', 'prevention', 'follow', 'journey']


In [15]:
dictionary = corpora.Dictionary(toked_tweets)

In [16]:
dictionary.save("snowball.tweets.dict")

In [17]:
print(dictionary)

Dictionary(10182 unique tokens: ['cells', 'combo', 'chemicals', 'prez', 'told']...)


In [18]:
gensim_corpus = [dictionary.doc2bow(tweet) for tweet in toked_tweets]#[[dictionary.token2id[t] for t in tweet] for tweet in toked_tweets]

In [19]:
lda = LdaModel(gensim_corpus, num_topics=20,
                    passes=10, alpha=0.001, id2word=dictionary)

In [20]:
lda.save("snowball-LDA-20-topics.model")

In [21]:
lda.show_topics()

['0.120*https + 0.066*rt + 0.050*st + 0.042*fightcancer + 0.041*co + 0.023*please + 0.021*periscope + 0.019*help + 0.019*amp + 0.015*vqdjndzrys',
 '0.066*pap + 0.039*rt + 0.038*congrats + 0.034*smear + 0.027*blessings + 0.025*amazing + 0.024*get + 0.023*mammogram + 0.021*test + 0.020*go',
 '0.183*msg + 0.144*crclub + 0.134*gurmeetramrahim + 0.120*rt + 0.045*fans + 0.021*love + 0.018*msgthefilm + 0.012*guruji + 0.008*saloniradhainsa + 0.008*gt',
 '0.078*doesn + 0.061*things + 0.045*incredible + 0.042*rt + 0.038*wid + 0.033*everywhere + 0.017*richards + 0.016*cecile + 0.015*also + 0.014*dhoom',
 '0.076*wearing + 0.034*thank + 0.029*moh + 0.029*made + 0.021*tomorrow + 0.021*make + 0.017*ur + 0.013*share + 0.013*rt + 0.013*part',
 '0.173*cancer + 0.130*http + 0.129*co + 0.068*cervical + 0.043*prevention + 0.036*rt + 0.029*breast + 0.027*screening + 0.022*reminder + 0.011*women',
 '0.079*rt + 0.040*within + 0.036*co + 0.036*celebrations + 0.036*http + 0.036*great + 0.035*ecrchat + 0.020*fee

In [22]:
for_viz = pyLDAvis.gensim.prepare(lda, gensim_corpus, dictionary)

/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:283: FutureWarning: order is deprecated, use sort_values(...)
  topic_proportion = (topic_freq / topic_freq.sum()).order(ascending=False)
/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:154: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  sort('saliency', ascending=False). \
/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:203: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  return token_table.sort(['Term', 'Topic'])


In [23]:
pyLDAvis.display(for_viz)

In [26]:
toked_tweets_by_tag, tags_to_raw_tweets = snowball.build_gensim_corpus(tweets, split_up_by_tag=True)

In [27]:
toked_tweets_by_tag.keys()

dict_keys(['mammogram', 'Gardasil', '#fightcancer', 'HPV vaccination', '#screened', 'other', 'pap smear', '#WomensHealth', 'HPV', '#stopcancerb4itstarts', 'colonoscopy', 'vaxx', '#vaccinated', 'cancer prevention', 'cervical cancer', '#GoingToTheDoctor'])

In [28]:
import importlib; importlib.reload(snowball)

<module 'snowball' from '/Users/byron/dev/snowball/data/snowball.py'>

In [29]:
lda_cervical, corpus_cervical, dict_cervical= snowball.gen_lda_model(toked_tweets_by_tag['cervical cancer'])

In [30]:
for_viz_cervical = pyLDAvis.gensim.prepare(lda_cervical, corpus_cervical, dict_cervical)

/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:283: FutureWarning: order is deprecated, use sort_values(...)
  topic_proportion = (topic_freq / topic_freq.sum()).order(ascending=False)
/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:154: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  sort('saliency', ascending=False). \
/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:203: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  return token_table.sort(['Term', 'Topic'])


In [31]:
pyLDAvis.display(for_viz_cervical)